In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-04-05 02:52:46--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-04-05 02:52:46 (47.2 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
import tensorflow as tf
tf.__version__

'2.8.0'

In [ ]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [ ]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"
unzip_data("nlp_getting_started.zip")

--2022-04-05 02:54:59--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.125.128, 142.250.157.128, 142.251.8.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.125.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2022-04-05 02:54:59 (95.4 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [ ]:
import pandas as pd
train_data=pd.read_csv("train.csv")
test_data=pd.read_csv("test.csv")
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train_shuffled= train_data.sample(frac=1,random_state=42)
train_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
train_data.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
print(f"Total training samples: {len(train_data)}")
print(f"Total test samples: {len(test_data)}")
print(f"Total samples: {len(train_data) + len(test_data)}")

Total training samples: 7613
Total test samples: 3263
Total samples: 10876


In [ ]:
import random
random_index = random.randint(0, len(train_data)-5)
for text,target in train_shuffled[["text", "target"]][random_index:random_index+5].iterrows():
  print(f"Target: {target[1]}", "(disaster)" if target[1] > 0 else "(not disaster)")
  print(f"Text:\n{target[0]}\n")
  print("\n")

Target: 1 (disaster)
Text:
California wildfires force thousands to evacuate: http://t.co/GFsl2Kwt5h via @YouTube



Target: 0 (not disaster)
Text:
Remove the http://t.co/Xxj2B4JxRt and Linkury Browser Hijacker http://t.co/9gtYlgXrOE http://t.co/yG6Rj86BKI



Target: 1 (disaster)
Text:
myGC: Broken powerlines evacuate Gold Coast tram suspend services http://t.co/6e7hHfeRz4



Target: 1 (disaster)
Text:
'The Reagan Administration had arranged for Israeli weapons to be sent to the Guatemalan Army  http://t.co/4fYNQ1hWWb



Target: 0 (not disaster)
Text:
#HSE releases annual workplace facilities data. Have a look | http://t.co/h4UshEekxm http://t.co/jNHNX3oISN





In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(train_shuffled["text"].to_numpy(),
                                                                            train_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [ ]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((6851,), (762,), (6851,), (762,))

In [ ]:
max_vocab_length = 10000
max_length = 15

text_vectorizer = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [ ]:
text_vectorizer.adapt(x_train)

In [ ]:
vocabulary=text_vectorizer.get_vocabulary()
top_5_words=vocabulary[:5]
bottom_5_words=vocabulary[-5:]
print(f"Number of words in vocab: {len(vocabulary)}")
print(f"Top 5 most common words: {top_5_words}") 
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [ ]:
tf.random.set_seed(42)
embedding=tf.keras.layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             embeddings_initializer="uniform",
                             input_length=max_length,
                             name="embedding") 
embedding

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()),
                    ("clf", MultinomialNB())
])
model_0.fit(x_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [ ]:
bayes_model=model_0.predict(x_test)
bayes_model

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,

In [ ]:
from helper_functions import calculate_results
bayes_results=calculate_results(bayes_model,y_test)

In [ ]:
bayes_results

{'accuracy': 79.26509186351706,
 'f1': 0.7990828614653861,
 'precision': 0.8336022277575122,
 'recall': 0.7926509186351706}

In [ ]:
from helper_functions import create_tensorboard_callback

SAVE_DIR = "model_logs"

In [ ]:
inputs = tf.keras.layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
#x=tf.keras.layers.Dense(32,activation='tanh')(x)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="Simple_DNN")

In [ ]:
def model_compiling(model):
  model.compile(loss="binary_crossentropy",
                optimizer='Adam',
                metrics=["accuracy"])
  return model.summary()

In [ ]:
model_compiling(model_1)

Model: "Simple_DNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_6   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_11 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-

In [ ]:
model_1_history = model_1.fit(x_train,
                              y_train,
                              epochs=10,
                              validation_data=(x_test,y_test),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="simple_DNN_Model")])

Saving TensorBoard log files to: model_logs/simple_DNN_Model/20220405-034259
Epoch 1/10
215/215 [==============================] - 4s 18ms/step - loss: 0.0436 - accuracy: 0.9812 - val_loss: 1.0732 - val_accuracy: 0.7533
Epoch 2/10
215/215 [==============================] - 3s 14ms/step - loss: 0.0420 - accuracy: 0.9804 - val_loss: 1.1070 - val_accuracy: 0.7507
Epoch 3/10
215/215 [==============================] - 3s 14ms/step - loss: 0.0411 - accuracy: 0.9812 - val_loss: 1.1306 - val_accuracy: 0.7546
Epoch 4/10
215/215 [==============================] - 3s 14ms/step - loss: 0.0406 - accuracy: 0.9823 - val_loss: 1.1617 - val_accuracy: 0.7507
Epoch 5/10
215/215 [==============================] - 3s 14ms/step - loss: 0.0397 - accuracy: 0.9819 - val_loss: 1.1870 - val_accuracy: 0.7559
Epoch 6/10
215/215 [==============================] - 3s 14ms/step - loss: 0.0401 - accuracy: 0.9815 - val_loss: 1.2087 - val_accuracy: 0.7559
Epoch 7/10
215/215 [==============================] - 3s 14ms/ste

In [ ]:
model_1_pred_probs = model_1.predict(x_test)
model_1_pred_probs[:10]

array([[7.0171344e-01],
       [6.8910372e-01],
       [9.9974722e-01],
       [2.0431745e-01],
       [4.4955063e-06],
       [9.9174845e-01],
       [8.2918227e-01],
       [1.0000000e+00],
       [1.0000000e+00],
       [9.4969344e-01]], dtype=float32)

In [ ]:
def model_predictions(model,x_test):
  model_pred_probs= model.predict(x_test)
  model_preds = tf.squeeze(tf.round(model_pred_probs))
  return model_preds

In [ ]:
model_1_pred=model_predictions(model_1,x_test)

In [ ]:
model_1_pred

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0.,
       0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1.,
       1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [ ]:
print('Simple DNN\n',
      calculate_results(y_test,model_1_pred))

Simple DNN
 {'accuracy': 75.45931758530183, 'precision': 0.7546642240189775, 'recall': 0.7545931758530183, 'f1': 0.7531886844350836}
